In [1]:
import mlflow
import matplotlib.pyplot as plt
from mlflow import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import pprint
import json
import ast
import sys
sys.path.insert(0, '/novo/projects/departments/cdd/public/users/ehec/au24/ML-BP-Domino')

from omegaconf import OmegaConf
from pathlib import Path
from hydra import compose, initialize, initialize_config_dir
from src.helpers.dataset import CSVDataLoader
from src.model.composite_model import CompositeModel
from torch.nn.utils.rnn import pad_sequence
import torch
from sklearn.metrics import matthews_corrcoef
from src.helpers.pytorch.light_attention import LightAttentionModule
import numpy as np
from sklearn.metrics import roc_auc_score
import os

2025-03-06 20:45:13.533754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 20:45:13.537111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 20:45:13.874872: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 20:45:14.523980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 20:45:26.340252: W tensorflow/compiler/tf2

In [2]:
%env PYTHONPATH="/novo/projects/departments/cdd/public/users/ehec/au24/ML-BP-Domino"

env: PYTHONPATH="/novo/projects/departments/cdd/public/users/ehec/au24/ML-BP-Domino"


In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = "KEY"
os.environ["AWS_SECRET_ACCESS_KEY"] = "KEY"
os.environ["AWS_SESSION_TOKEN"] = "KEY"

In [4]:
!/nfs_home/users/ehec/miniconda3/condabin/conda run --no-capture-output -n su24 TORCH_USE_CUDA_DSA=1 CUDA_LAUNCH_BLOCKING=1 python /novo/projects/departments/cdd/public/users/ehec/au24/ML-BP-Domino/src/cli/training.py "mlflow=datalab" "persistence=lovelace" "persistence.training_data=fibrillation_data" "dataset.use_predefined_split=true" "dataset.data_name=sbxw_fibrillation_peptide_waltzdb-train-val_serrano-test_randpadded10x_padlen-10" "dataset.task=classification_binary" "persistence.artifacts_root_folder=/novo/projects/departments/cdd/public/users/ehec/au24/250218_respred_dump" "embedder=esm2_t6_8M" "embedder.mean_pool=false" "dimred=noreduction" "predictor=lightattention" "predictor.hparams.kernel_size=5" "predictor.hparams.post_attention=mlp" "predictor.hparams.reduction_mode=weighted_mean_two" "predictor.hparams.max_epochs=500" "predictor.hparams.patience=200" "predictor.hparams.batch_size=1000" "predictor.hparams.learning_rate=0.005" "mlflow.experiment_name=ML-BP-Domino" "plots.generate_plots=false"

2025-03-06 20:47:11.884412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 20:47:11.884457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 20:47:11.885905: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 20:47:11.893081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-06 20:47:15.280370: W tensorflow/compiler/tf2